#  Credit scoring research

 Pandas library import. Now let's read the table `data.csv` and import it in a dataframe `data` 

In [1]:
import pandas as pd

data = pd.read_csv('data.csv')

Quick look at the first ten lines

In [2]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


and get some basic information:

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Data preprocessing

### Missing values removal
Let's find the number of missing values for each column:

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In two columns, there are missing values: `days_employed` indicating the lenghts of the current place of employement, and  `total_income`   which stores income data. The income amount is strongly influenced by employment type, so the missing values in this column will be filled with the median value for each `income_type` category. For example, for an individual with an employment type of "employee," the missing value in the `total_income` column will be filled with the median income among all records with the same employment type.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

In [6]:
### check if it worked
data['total_income'].isna().sum()

0

### Processing abnormal values

There may be artifacts (anomalies) in the data, which are values that do not reflect reality and appeared due to some kind of error. An example of such an artifact in our data is a negative number of days in the `days_employed` column. We can process the values in this column by replacing all negative values with their absolute values using the abs() method.

In [7]:
data['days_employed'] = data['days_employed'].abs()

Let's check the median value of the employment duration (`days_employed`) for each employment type (in days).

Please note, that employment types are given in Russian.

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

For two types (безработный - unemployed and пенсионер - retirees), there are abnormally large values. It is difficult to correct such values, so we prefer to leave them as they are. Moreover, we hardly need this column for this analysis.

Let's get the list of unique values in `children` :


In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

We have two anomalies: '-1' and '20'. Let's get rid of them

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

And now check the result:

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Now we can fill in the missing values in `days_employed` column with the median values for each `income_type` category.

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

All gone, great.

### Data type conversion

Let's change the data type of `total_income` from floating-point to integer using the astype() method.

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Duplicate handling

Let's handle implicit duplicates in the `education` column. In this column, there are the same values but written differently, using uppercase and lowercase letters. We will convert them to lowercase and check the other columns as well.

In [15]:
data['education'] = data['education'].str.lower()

Now we can count the duplicates:

In [16]:
data.duplicated().sum()

71

And delete them

In [17]:
data = data.drop_duplicates()


Let's create a column named `total_income_category` in the data dataframe with the following categories:

- 0-30000: 'E'
- 30001-50000: 'D'
- 50001-200000: 'C'
- 200001-1000000: 'B'
- 1000001 and above: 'A'
For example, a credit recipient with an income of 25000 rubles will be assigned the category 'E', while a client earning 235000 will be assigned the category 'B'. 

Let's declare the categorize_income function

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Let's print the list of unique loan purposes from the purpose column on the screen.

Please note, that values are given in Russian.

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Now let's create a function that will generate a new column named purpose_category based on the data from the purpose column. The purpose_category column will include the following categories:

- 'Car operations'
- 'Real estate operations'
- 'Wedding planning'
- 'College funding'

For example, if the purpose column contains the substring 'car purchase' ('приобретение автомобиля'), the purpose_category column should have the value 'Car operations'.

We will apply our own function categorize_purpose() using the apply() method. We'll also evaluate the data in the purpose column to determine which substrings will help us choose the appropriate category.

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'Car operations'
        elif 'жил' in row or 'недвиж' in row:
            return 'Real estate operations'
        elif 'свад' in row:
            return 'Wedding planning'
        elif 'образов' in row:
            return 'College funding'
    except:
        return 'No category'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

And now let's take a look at the dataframe again:

In [23]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,Real estate operations
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,Car operations
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,Real estate operations
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,College funding
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,Wedding planning


## Data analysis
### Is there a correlation between the number of children and loan repayment on time?

Let's group the data by the number of children among the borrowers who have outstanding debts.



In [24]:
data.loc[data['debt']==1].groupby('children')['debt'].count().to_frame()

,debt
children,
0,1063
1,444
2,194
3,27
4,4


The proportion of individuals with 2 or more children among borrowers with outstanding debts is indeed lower. Let's now examine the dependency on the total number of bank clients.

Let's create a dataframe that reflects the relationship between the number of children and the occurrence of overdue debts.

In [25]:
debt_parential_status=pd.DataFrame()
#create a column where we will group cases of overdue debts by the number of children of the client
debt_parential_status['sum_parential_status'] = data.groupby('children')['debt'].sum()

Let's create a column where we will group clients by the number of children and calculate the proportion of cases with overdue debts compared to the total number of clients in each category.

In [26]:
debt_parential_status['count_parential_status'] = data.groupby('children')['debt'].count()
debt_parential_status['result_parential_status'] = debt_parential_status['sum_parential_status'] / debt_parential_status['count_parential_status'] 
debt_parential_status.head()

,sum_parential_status,count_parential_status,result_parential_status
children,,,
0,1063,14091,0.075438
1,444,4808,0.092346
2,194,2052,0.094542
3,27,330,0.081818
4,4,41,0.097561


Conclusion: The proportion of clients with overdue debts is slightly higher among parents compared to childless clients by almost two percent. It is worth noting that clients with four children have no overdue debts, although the number of observations in this category (41) is not sufficient for a significant conclusion.

#### Is there a correlation between marital status and loan repayment on time?



Let's create a dataframe `debt_maritual_status` that reflects the relationship between marital status and the occurrence of overdue debts. We'll create a column where we group cases of overdue debts by marital status and another column where we group clients by marital status. Then, we'll calculate the proportion of cases with overdue debts compared to the total number of clients in each category.

Please note, the maritual status is given in Russian.

In [27]:
debt_maritual_status=pd.DataFrame()
debt_maritual_status['sum_maritual_status'] = data.groupby('family_status')['debt'].sum()
debt_maritual_status['count_maritual_status'] = data.groupby('family_status')['debt'].count()
debt_maritual_status['result_maritual_status'] = debt_maritual_status['sum_maritual_status'] / debt_maritual_status['count_maritual_status'] 
debt_maritual_status.head()

,sum_maritual_status,count_maritual_status,result_maritual_status
family_status,,,
Не женат / не замужем,273,2796,0.097639
в разводе,84,1189,0.070648
вдовец / вдова,63,951,0.066246
гражданский брак,385,4134,0.093130
женат / замужем,927,12261,0.075606


Conclusion: Based on the available research tools, it can be suggested that there is a positive correlation between marital status and loan repayment on time. Individuals who are married (*'женат/замужем'*) or have been married (*'вдовец/вдова'* or *'в разводе'*) are less likely to have cases of overdue debts.

#### Is there a correlation between income level and loan repayment on time?

Again let's create a dataframe `debt_income_status` that reflects the relationship between income level and the occurrence of overdue debts. We'll create a column where we group cases of overdue debts by income category and another column where we group clients by income category. Then, we'll calculate the proportion of cases with overdue debts compared to the total number of clients in each income category.

In [28]:
debt_income_status=pd.DataFrame()
debt_income_status['sum_income_status'] = data.groupby('total_income_category')['debt'].sum()
debt_income_status['count_income_status'] = data.groupby('total_income_category')['debt'].count()
debt_income_status['result_income_status'] = debt_income_status['sum_income_status'] / debt_income_status['count_income_status'] 
debt_income_status.head()

,sum_income_status,count_income_status,result_income_status
total_income_category,,,
A,2,25,0.080000
B,354,5014,0.070602
C,1353,15921,0.084982
D,21,349,0.060172
E,2,22,0.090909


Conclusion:
No clear correlation between income level and loan repayment on time has been observed.

#### How do different loan purposes impact loan repayment on time?

And again we create a dataframe that reflects the relationship between loan purposes and the occurrence of overdue debts. We'll create a column where we group cases of overdue debts by loan purposes and another column where we group clients by loan purposes. Then, we'll calculate the proportion of cases with overdue debts compared to the total number of clients in each loan purpose category.

In [29]:
debt_purpose_category=pd.DataFrame()
debt_purpose_category['sum_purpose_category'] = data.groupby('purpose_category')['debt'].sum()
debt_purpose_category['count_purpose_category'] = data.groupby('purpose_category')['debt'].count()
debt_purpose_category['result_purpose_category'] = debt_purpose_category['sum_purpose_category'] / debt_purpose_category['count_purpose_category'] 
debt_purpose_category.head()

,sum_purpose_category,count_purpose_category,result_purpose_category
purpose_category,,,
Car operations,400,4279,0.093480
College funding,369,3988,0.092528
Real estate operations,780,10751,0.072551
Wedding planning,183,2313,0.079118


Conclusion: the loan purposes of car loans and loans for education appear to be more problematic, with a higher proportion of clients having overdue payments, exceeding 9%. On the other hand, mortgages seem to be the safest category, with a lower proportion of problematic clients at 7.2%



## Final conclusion

Based on the results of the study regarding the relationship between credit scoring and loan repayment on time, the following findings can be observed:

There is a notable correlation between the number of children and loan repayment. Clients with more children tend to have a slightly lower proportion of overdue payments.

Marital status also plays a role in loan repayment. Clients who are married or have previously been married (widowed, divorced) demonstrate a lower proportion of overdue payments compared to unmarried clients.

The impact of income level on loan repayment is not evident. There is no clear association between income level and the likelihood of timely loan repayment.

The purpose of the loan influences loan repayment. Car loans and loans for education exhibit a higher proportion of clients with overdue payments, while mortgages appear to be the least risky category with a lower proportion of problematic clients.

These findings can provide valuable insights for credit scoring models and loan assessment processes, allowing lenders to better evaluate the risk of loan default and make informed decisions regarding loan approvals and interest rates. It is important to note that further analysis and consideration of additional factors may be necessary to enhance the accuracy of credit scoring models and minimize the risk of loan defaults.